In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from icecream import ic
from sklearn.preprocessing import LabelEncoder
import time
from itertools import product
from icecream import ic


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

sales_train = pd.read_csv('./data/sales_train.csv')
test = pd.read_csv('./data/test.csv')   # (214200, 3)

# 计算每个商品每个月的销售量，假如某个商品在某个月没有数据，则填充0（即这个月的销售量为0）
sales_by_item_id = sales_train.pivot_table(index=['item_id'], values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
sales_by_item_id.columns = sales_by_item_id.columns.droplevel().map(str)
sales_by_item_id.columns.values[0] = 'item_id'
sales_by_item_id = sales_by_item_id.rename_axis(None, axis=1)

# 获取最近6个月销售量为0的数据
# six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
# six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表
# test.loc[test.item_id.isin(six_zero_item_id), 'item_cnt_month'] = 0  # 将test数据中（最近六个月销量为0）的数据月销量设为0，有7812个

# 计算每个商店每个月的销量
sales_by_shop_id = sales_train.pivot_table(index=['shop_id'], values=['item_cnt_day'], aggfunc=np.sum, fill_value=0, columns='date_block_num').reset_index()
sales_by_shop_id.columns = sales_by_shop_id.columns.droplevel().map(str)    # 将两层column转化为一层column,保留下层column
sales_by_shop_id.columns.values[0] = 'shop_id'
sales_by_shop_id = sales_by_shop_id.rename_axis(None, axis=1)   # 将列方向的轴重命名为none

# zero = sales_train[sales_train.date_block_num==0]
# ic(zero.shop_id.unique(), len(zero.item_id.unique()), len(zero.shop_id.unique()), len(zero.shop_id.unique()) * len(zero.item_id.unique()))
# ic(sales_train.shop_id.unique(), len(sales_train.item_id.unique()), len(sales_train.shop_id.unique()), len(sales_train.shop_id.unique()) * len(sales_train.item_id.unique()))

"""组合date_block_num,shop_id,item_id(部分) 总量：10913850"""
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = sales_train[sales_train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix.sort_values(cols, inplace=True)  # 排序
sales_train['revenue'] = sales_train['item_price'] * sales_train['item_cnt_day']    # 某一天的销售额

# 分组
groupby = sales_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_cnt_day': 'sum'}).reset_index()
groupby = groupby.rename(columns={'item_cnt_day': 'item_cnt_month'})
matrix = matrix.merge(groupby, on=['date_block_num','shop_id','item_id'], how='left')
matrix['item_cnt_month'] = matrix['item_cnt_month'].fillna(0).clip(0, 20)
matrix['item_cnt_month'] = matrix['item_cnt_month'].astype(np.float16)

# test数据
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

# 合并matrix,test
matrix = pd.concat([matrix, test[cols]], ignore_index=True, axis=0)
matrix['item_cnt_month'].fillna(0, inplace=True)

# 商品信息
items = pd.read_csv('./data/items.csv')
items = items[['item_id', 'item_category_id']]
matrix = pd.merge(left=matrix, right=items, on='item_id', how='left')  # 合并

# 商品类别
le = LabelEncoder()
categories = pd.read_csv('./data/item_categories.csv')
categories['split'] = categories['item_category_name'].str.split('-')
categories['type'] = categories['split'].map(lambda x:x[0].strip())
categories['subtype'] = categories['split'].map(lambda x:x[1].strip() if len(x)>1 else x[0].strip())
categories = categories[['item_category_id','type','subtype']]
categories['cat_type_code'] = le.fit_transform(categories['type'])
categories['cat_subtype_code'] = le.fit_transform(categories['subtype'])
matrix = pd.merge(left=matrix, right=categories[['item_category_id','cat_type_code','cat_subtype_code']], on='item_category_id', how='left')    # 合并

# 商店信息
shops = pd.read_csv('./data/shops.csv')
shops['split']=shops.shop_name.str.split(' ')
shops['shop_city'] = shops['split'].map(lambda x:x[0])
shops['shop_city_code'] = le.fit_transform(shops['shop_city'])

def st(name):
    ic(name)
    if 'ТЦ' in name or 'ТРЦ' in name:
        shopt = 'ТЦ'
    elif 'ТК' in name:
        shopt = 'ТК'
    elif 'ТРК' in name:
        shopt = 'ТРК'
    elif 'МТРЦ' in name:
        shopt = 'МТРЦ'
    else:
        shopt = 'UNKNOWN'
    return shopt
shops['shop_type'] = shops['shop_name'].apply(st)

shops.loc[shops.shop_id == 21, 'shop_type'] = 'МТРЦ'   # 修正
shops['shop_type_code'] = le.fit_transform(shops['shop_type'])
matrix = pd.merge(left=matrix, right=shops[['shop_id','shop_city_code','shop_type_code']], on='shop_id', how='left')    # 合并
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['cat_type_code'] = matrix['cat_type_code'].astype(np.int8)
matrix['cat_subtype_code'] = matrix['cat_subtype_code'].astype(np.int8)
matrix['shop_city_code'] = matrix['shop_city_code'].astype(np.int8)
matrix['shop_type_code'] = matrix['shop_type_code'].astype(np.int8)


"""历史信息"""

def lag_features(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(left=df, right=shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

# 月销量（所有商品）
group = matrix.groupby('date_block_num').agg({'item_cnt_month': 'mean'}).reset_index()
group.columns = ['date_block_num', 'date_avg_item_cnt']
matrix = pd.merge(left=matrix, right=group, on='date_block_num', how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_avg_item_cnt')
matrix.drop('date_avg_item_cnt', axis=1, inplace=True)

# 月销量（每一件商品）
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop('date_item_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个商店 ）
group = matrix.groupby(['date_block_num','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop('date_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（每个类别）
group = matrix.groupby(['date_block_num','item_category_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_avg_item_cnt')
matrix.drop('date_cat_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品类别-商店）
group = matrix.groupby(['date_block_num','item_category_id','shop_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_cat_shop_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','item_category_id','shop_id'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_cat_shop_avg_item_cnt')
matrix.drop('date_cat_shop_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品大类）
group = matrix.groupby(['date_block_num','cat_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_type_avg_item_cnt')
matrix.drop('date_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商品大类） ++++++++++++ 和 月销量（商品）是重复的，因为每一个商品，类别是确定的，大类也是确定的
group = matrix.groupby(['date_block_num', 'item_id', 'cat_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_type_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'cat_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_type_avg_item_cnt')
matrix.drop('date_item_type_avg_item_cnt', axis=1, inplace=True)

# 月销量（商店城市）
group = matrix.groupby(['date_block_num','shop_city_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_city_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_city_avg_item_cnt')
matrix.drop('date_city_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店城市）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_city_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_city_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_city_avg_item_cnt')
matrix.drop('date_item_city_avg_item_cnt', axis=1, inplace=True)

# 趋势特征
group = sales_train.groupby('item_id').agg({'item_price': 'mean'})
group.columns = ['item_avg_item_price']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = sales_train.groupby(['date_block_num','item_id']).agg({'item_price': 'mean'})
group.columns = ['date_item_avg_item_price']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'date_item_avg_item_price')
for i in lags:
    matrix['delta_price_lag_'+str(i)]=(matrix['date_item_avg_item_price_lag_' + str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if pd.notnull(row['delta_price_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_price_lag_'+str(i)]
    return 0

matrix['delta_price_lag']=matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)

features_to_drop = ['item_avg_item_price','date_item_avg_item_price']
for i in lags:
    features_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    features_to_drop += ['delta_price_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

# 每个月的天数
matrix['month'] = matrix['date_block_num'] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix['days'] = matrix['month'].map(days)
matrix['days'] = matrix['days'].astype(np.int8)

# 开始销量
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

ic| name: '!Якутск Орджоникидзе, 56 фран'
ic| name: '!Якутск ТЦ "Центральный" фран'
ic| name: 'Адыгея ТЦ "Мега"'
ic| name: 'Балашиха ТРК "Октябрь-Киномир"'
ic| name: 'Волжский ТЦ "Волга Молл"'
ic| name: 'Вологда ТРЦ "Мармелад"'
ic| name: 'Воронеж (Плехановская, 13)'
ic| name: 'Воронеж ТРЦ "Максимир"'
ic| name: 'Воронеж ТРЦ Сити-Парк "Град"'
ic| name: 'Выездная Торговля'
ic| name: 'Жуковский ул. Чкалова 39м?'
ic| name: 'Жуковский ул. Чкалова 39м²'
ic| name: 'Интернет-магазин ЧС'
ic| name: 'Казань ТЦ "Бехетле"'
ic| name: 'Казань ТЦ "ПаркХаус" II'
ic| name: 'Калуга ТРЦ "XXI век"'
ic| name: 'Коломна ТЦ "Рио"'
ic| name: 'Красноярск ТЦ "Взлетка Плаза"'
ic| name: 'Красноярск ТЦ "Июнь"'
ic| name: 'Курск ТЦ "Пушкинский"'
ic| name: 'Москва "Распродажа"'
ic| name: 'Москва МТРЦ "Афи Молл"'
ic| name: 'Москва Магазин С21'
ic| name: 'Москва ТК "Буденовский" (пав.А2)'
ic| name: 'Москва ТК "Буденовский" (пав.К7)'
ic| name: 'Москва ТРК "Атриум"'
ic| name: 'Москва ТЦ "Ареал" (Беляево)'
ic| name: 'Москва 

In [95]:
# 获取最近6个月销售量为0的数据
six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表

In [177]:
matrix.shape

(11128050, 59)

In [6]:
# matrix_11128050_59 = matrix.copy()

In [175]:
matrix = matrix_11128050_59.copy()

In [176]:
matrix.shape

(11128050, 59)

In [178]:
matrix = lag_features(matrix, [1,2,3,6,12], 'item_cnt_month')

In [179]:
matrix.shape

(11128050, 64)

In [180]:
matrix.columns

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'cat_subtype_code',
       'shop_city_code', 'shop_type_code', 'date_avg_item_cnt_lag_1',
       'date_avg_item_cnt_lag_2', 'date_avg_item_cnt_lag_3',
       'date_avg_item_cnt_lag_6', 'date_avg_item_cnt_lag_12',
       'date_item_avg_item_cnt_lag_1', 'date_item_avg_item_cnt_lag_2',
       'date_item_avg_item_cnt_lag_3', 'date_item_avg_item_cnt_lag_6',
       'date_item_avg_item_cnt_lag_12', 'date_shop_avg_item_cnt_lag_1',
       'date_shop_avg_item_cnt_lag_2', 'date_shop_avg_item_cnt_lag_3',
       'date_shop_avg_item_cnt_lag_6', 'date_shop_avg_item_cnt_lag_12',
       'date_cat_avg_item_cnt_lag_1', 'date_cat_avg_item_cnt_lag_2',
       'date_cat_avg_item_cnt_lag_3', 'date_cat_avg_item_cnt_lag_6',
       'date_cat_avg_item_cnt_lag_12', 'date_cat_shop_avg_item_cnt_lag_1',
       'date_cat_shop_avg_item_cnt_lag_2', 'date_cat_shop_avg_item_cnt_lag_3',
       'date_cat_shop_avg_ite

In [181]:
matrix.drop(['date_item_type_avg_item_cnt_lag_1',
       'date_item_type_avg_item_cnt_lag_2',
       'date_item_type_avg_item_cnt_lag_3',
       'date_item_type_avg_item_cnt_lag_6',
       'date_item_type_avg_item_cnt_lag_12'], axis=1, inplace=True)

In [182]:
matrix.shape

(11128050, 59)

In [183]:
# 月销量（商店类型）
group = matrix.groupby(['date_block_num','shop_type_code']).agg({'item_cnt_month': 'mean'})
group.columns = ['date_shoptype_avg_item_cnt']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['date_block_num','shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_shoptype_avg_item_cnt')
matrix.drop('date_shoptype_avg_item_cnt', axis=1, inplace=True)

# 月销量（商品-商店类型）
group = matrix.groupby(['date_block_num', 'item_id', 'shop_type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_item_shoptype_avg_item_cnt']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'item_id', 'shop_type_code'], how='left')
matrix = lag_features(matrix, [1,2,3,6,12], 'date_item_shoptype_avg_item_cnt')
matrix.drop('date_item_shoptype_avg_item_cnt', axis=1, inplace=True)

# # 月销量（商店-商品）
# group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
# group.columns = [ 'date_shopitem_avg_item_cnt' ]
# group.reset_index(inplace=True)
# matrix = pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')
# matrix = lag_features(matrix, [1,2,3,6,12], 'date_shopitem_avg_item_cnt')
# matrix.drop('date_shopitem_avg_item_cnt', axis=1, inplace=True)

In [184]:
matrix.shape

(11128050, 69)

In [31]:
matrix.loc[matrix.date_block_num==34, 'item_cnt_month'] = np.nan

In [34]:
matrix[matrix.date_block_num == 34]

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,cat_subtype_code,shop_city_code,shop_type_code,date_avg_item_cnt_lag_1,...,date_shoptype_avg_item_cnt_lag_1,date_shoptype_avg_item_cnt_lag_2,date_shoptype_avg_item_cnt_lag_3,date_shoptype_avg_item_cnt_lag_6,date_shoptype_avg_item_cnt_lag_12,date_item_shoptype_avg_item_cnt_lag_1,date_item_shoptype_avg_item_cnt_lag_2,date_item_shoptype_avg_item_cnt_lag_3,date_item_shoptype_avg_item_cnt_lag_6,date_item_shoptype_avg_item_cnt_lag_12
10913850,34,5,5037,NaN,19,5,10,4,4,0.258545,...,0.220703,0.230591,0.275146,0.244873,0.30835,0.620605,2.666016,3.310547,2.162109,1.441406
10913851,34,5,5320,NaN,55,13,2,4,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10913852,34,5,5233,NaN,19,5,10,4,4,0.258545,...,0.220703,0.230591,0.275146,0.244873,NaN,1.172852,1.900391,3.828125,1.806641,NaN
10913853,34,5,5232,NaN,23,5,16,4,4,0.258545,...,0.220703,0.230591,0.275146,NaN,NaN,0.862305,1.200195,1.655273,NaN,NaN
10913854,34,5,5268,NaN,20,5,11,4,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11128045,34,45,18454,NaN,55,13,2,21,4,0.258545,...,0.220703,0.230591,0.275146,0.244873,NaN,0.068970,0.033325,0.034485,0.548340,NaN
11128046,34,45,16188,NaN,64,14,42,21,4,0.258545,...,0.220703,0.230591,NaN,NaN,NaN,0.000000,0.099976,NaN,NaN,NaN
11128047,34,45,15757,NaN,55,13,2,21,4,0.258545,...,0.220703,0.230591,0.275146,0.244873,0.30835,0.137939,0.066650,0.137939,0.129028,0.205933
11128048,34,45,19648,NaN,40,11,4,21,4,0.258545,...,0.220703,0.230591,0.275146,0.244873,NaN,0.034485,0.033325,0.172363,0.096802,NaN


In [35]:
matrix.shape

(11128050, 69)

In [36]:
# **********趋势特征 delta_cnt_month_lag ********************
group = matrix.groupby('item_id').agg({'item_cnt_month': 'mean'})
group.columns = ['trend_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on='item_id', how='left')

group = matrix.groupby(['date_block_num','item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['trend_date_item_avg_cnt_month']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num','item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'trend_date_item_avg_cnt_month')
for i in lags:
    matrix['delta_cnt_month_lag_'+str(i)] = (matrix['trend_date_item_avg_cnt_month_lag_' + str(i)] - matrix['trend_item_avg_cnt_month']) / matrix['trend_item_avg_cnt_month']

def select_trend2(row):
    for i in lags:
        if pd.notnull(row['delta_cnt_month_lag_'+str(i)]):  # 如果不是NaN
            return row['delta_cnt_month_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta_cnt_month_lag'] = matrix.apply(select_trend2, axis=1)
matrix['delta_cnt_month_lag'] = matrix['delta_cnt_month_lag'].astype(np.float16)

features_to_drop = ['trend_item_avg_cnt_month','trend_date_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['trend_date_item_avg_cnt_month_lag_'+str(i)]
    features_to_drop += ['delta_cnt_month_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

In [37]:
matrix.shape

(11128050, 70)

In [38]:
# **********趋势特征 delta2_cnt_month_lag ********************
group = matrix.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_shop_item_avg_cnt_month']
group = group.reset_index()
matrix = pd.merge(left=matrix, right=group, on=['shop_id', 'item_id'], how='left')

group = matrix.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_month': 'mean'})
group.columns = ['qushi_date_shop_item_avg_cnt_month']
group = group.reset_index()
matrix=pd.merge(left=matrix, right=group, on=['date_block_num', 'shop_id', 'item_id'], how='left')

# 计算matrix中商品的历史价格
lags = [1,2,3,4,5,6,12]
matrix = lag_features(matrix, lags, 'qushi_date_shop_item_avg_cnt_month')
for i in lags:
    matrix['delta2_cnt_month_lag_'+str(i)] = (matrix['qushi_date_shop_item_avg_cnt_month_lag_' + str(i)] - matrix['qushi_shop_item_avg_cnt_month']) / matrix['qushi_shop_item_avg_cnt_month']

def select_trend3(row):
    for i in lags:
        if pd.notnull(row['delta2_cnt_month_lag_'+str(i)]):  # 如果不是NaN
            return row['delta2_cnt_month_lag_'+str(i)]
    return 0   #  如果delta_price_lag_都为空，那么将趋势设为0，0代表没有趋势

matrix['delta2_cnt_month_lag'] = matrix.apply(select_trend3, axis=1)
matrix['delta2_cnt_month_lag'] = matrix['delta2_cnt_month_lag'].astype(np.float16)

features_to_drop = ['qushi_shop_item_avg_cnt_month','qushi_date_shop_item_avg_cnt_month']
for i in lags:
    features_to_drop += ['delta2_cnt_month_lag_'+str(i)]
matrix.drop(features_to_drop, axis=1, inplace=True)

In [41]:
matrix.shape

(11128050, 78)

In [40]:
# matrix_11128050_78_bak = matrix.copy()

In [157]:
matrix.shape

(11128050, 64)

In [159]:
dict(matrix.count())

{'date_block_num': 11128050,
 'shop_id': 11128050,
 'item_id': 11128050,
 'item_cnt_month': 11128050,
 'item_category_id': 11128050,
 'cat_type_code': 11128050,
 'cat_subtype_code': 11128050,
 'shop_city_code': 11128050,
 'shop_type_code': 11128050,
 'date_avg_item_cnt_lag_1': 8605461,
 'date_avg_item_cnt_lag_2': 7953134,
 'date_avg_item_cnt_lag_3': 7344764,
 'date_avg_item_cnt_lag_6': 5711598,
 'date_avg_item_cnt_lag_12': 3120910,
 'date_item_avg_item_cnt_lag_1': 8605461,
 'date_item_avg_item_cnt_lag_2': 7953134,
 'date_item_avg_item_cnt_lag_3': 7344764,
 'date_item_avg_item_cnt_lag_6': 5711598,
 'date_item_avg_item_cnt_lag_12': 3120910,
 'date_shop_avg_item_cnt_lag_1': 8605461,
 'date_shop_avg_item_cnt_lag_2': 7953134,
 'date_shop_avg_item_cnt_lag_3': 7344764,
 'date_shop_avg_item_cnt_lag_6': 5711598,
 'date_shop_avg_item_cnt_lag_12': 3120910,
 'date_cat_avg_item_cnt_lag_1': 8605461,
 'date_cat_avg_item_cnt_lag_2': 7953134,
 'date_cat_avg_item_cnt_lag_3': 7344764,
 'date_cat_avg_item

In [166]:
# 新--删除前12个月数据
ts = time.time()

# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)
time.time() - ts

3.3927719593048096

In [42]:
# 因为有12个月的延迟特征（1，2，3，6，12）（1，2，3，4，5，6，12），所以需要删除前12月的数据
matrix = matrix[matrix['date_block_num'] > 11]

# 找到有NaN值的列，然后把那些列中的NaN值填充0
columns = matrix.columns
column_null = []
for i in columns:
    if len(matrix[matrix[i].isnull()]) > 0:
        column_null.append(i)

for i in column_null:
    matrix[i].fillna(0, inplace=True)

In [167]:
matrix.shape

(6639294, 64)

In [168]:
dict(matrix.count())

{'date_block_num': 6639294,
 'shop_id': 6639294,
 'item_id': 6639294,
 'item_cnt_month': 6639294,
 'item_category_id': 6639294,
 'cat_type_code': 6639294,
 'cat_subtype_code': 6639294,
 'shop_city_code': 6639294,
 'shop_type_code': 6639294,
 'date_avg_item_cnt_lag_1': 6639294,
 'date_avg_item_cnt_lag_2': 6639294,
 'date_avg_item_cnt_lag_3': 6639294,
 'date_avg_item_cnt_lag_6': 6639294,
 'date_avg_item_cnt_lag_12': 6639294,
 'date_item_avg_item_cnt_lag_1': 6639294,
 'date_item_avg_item_cnt_lag_2': 6639294,
 'date_item_avg_item_cnt_lag_3': 6639294,
 'date_item_avg_item_cnt_lag_6': 6639294,
 'date_item_avg_item_cnt_lag_12': 6639294,
 'date_shop_avg_item_cnt_lag_1': 6639294,
 'date_shop_avg_item_cnt_lag_2': 6639294,
 'date_shop_avg_item_cnt_lag_3': 6639294,
 'date_shop_avg_item_cnt_lag_6': 6639294,
 'date_shop_avg_item_cnt_lag_12': 6639294,
 'date_cat_avg_item_cnt_lag_1': 6639294,
 'date_cat_avg_item_cnt_lag_2': 6639294,
 'date_cat_avg_item_cnt_lag_3': 6639294,
 'date_cat_avg_item_cnt_lag_

# 分析

# 训练

In [49]:
matrix.shape

(6639294, 78)

In [50]:
"""建模"""
trainData = matrix[matrix['date_block_num'] < 33]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 33]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)333

In [53]:
import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 2000,
    'max_depth': 8,
    'num_leaves': 200,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.01,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.318419 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13137
[LightGBM] [Info] Number of data points in the train set: 6186922, number of used features: 77
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.288852
[1]	training's rmse: 1.18307	valid_1's rmse: 1.13238
[2]	training's rmse: 1.17795	valid_1's rmse: 1.12848
[3]	training's rmse: 1.17329	valid_1's rmse: 1.12514
[4]	training's rmse: 1.16853	valid_1's rmse: 1.12178
[5]	training's rmse: 1.16373	valid_1's rmse: 1.11823
[6]	training's rmse: 1.15924	valid_1's rmse: 1.11468
[7]	training's rmse: 1.15455	valid_1's rmse: 1.11115
[8]	training's rmse: 1.14977	valid_1's rmse

[139]	training's rmse: 0.877786	valid_1's rmse: 0.927022
[140]	training's rmse: 0.876935	valid_1's rmse: 0.926727
[141]	training's rmse: 0.876215	valid_1's rmse: 0.926457
[142]	training's rmse: 0.875169	valid_1's rmse: 0.925975
[143]	training's rmse: 0.874518	valid_1's rmse: 0.925705
[144]	training's rmse: 0.873733	valid_1's rmse: 0.925395
[145]	training's rmse: 0.872748	valid_1's rmse: 0.924811
[146]	training's rmse: 0.871972	valid_1's rmse: 0.924463
[147]	training's rmse: 0.871261	valid_1's rmse: 0.924296
[148]	training's rmse: 0.870663	valid_1's rmse: 0.923983
[149]	training's rmse: 0.869986	valid_1's rmse: 0.923651
[150]	training's rmse: 0.869013	valid_1's rmse: 0.923162
[151]	training's rmse: 0.868262	valid_1's rmse: 0.92277
[152]	training's rmse: 0.867469	valid_1's rmse: 0.922539
[153]	training's rmse: 0.866975	valid_1's rmse: 0.922292
[154]	training's rmse: 0.866276	valid_1's rmse: 0.922054
[155]	training's rmse: 0.865459	valid_1's rmse: 0.921614
[156]	training's rmse: 0.86484	v

[281]	training's rmse: 0.810584	valid_1's rmse: 0.906901
[282]	training's rmse: 0.810298	valid_1's rmse: 0.906981
[283]	training's rmse: 0.809831	valid_1's rmse: 0.906758
[284]	training's rmse: 0.809662	valid_1's rmse: 0.906806
[285]	training's rmse: 0.809469	valid_1's rmse: 0.906959
[286]	training's rmse: 0.809231	valid_1's rmse: 0.907086
[287]	training's rmse: 0.808949	valid_1's rmse: 0.906933
[288]	training's rmse: 0.808767	valid_1's rmse: 0.906945
[289]	training's rmse: 0.80837	valid_1's rmse: 0.90691
[290]	training's rmse: 0.808121	valid_1's rmse: 0.906832
[291]	training's rmse: 0.807961	valid_1's rmse: 0.90686
[292]	training's rmse: 0.807698	valid_1's rmse: 0.906826
[293]	training's rmse: 0.80755	valid_1's rmse: 0.906919
[294]	training's rmse: 0.807318	valid_1's rmse: 0.906799
[295]	training's rmse: 0.807139	valid_1's rmse: 0.906952
[296]	training's rmse: 0.806899	valid_1's rmse: 0.906954
[297]	training's rmse: 0.80666	valid_1's rmse: 0.90691
[298]	training's rmse: 0.80651	valid_

[404]	training's rmse: 0.789076	valid_1's rmse: 0.908481
[405]	training's rmse: 0.788985	valid_1's rmse: 0.908432
[406]	training's rmse: 0.788836	valid_1's rmse: 0.908456
[407]	training's rmse: 0.788727	valid_1's rmse: 0.908567
[408]	training's rmse: 0.788522	valid_1's rmse: 0.908553
[409]	training's rmse: 0.788447	valid_1's rmse: 0.908543
[410]	training's rmse: 0.788279	valid_1's rmse: 0.908606
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[411]	training's rmse: 0.788175	valid_1's rmse: 0.908688
[412]	training's rmse: 0.78806	valid_1's rmse: 0.908705
[413]	training's rmse: 0.787949	valid_1's rmse: 0.908749
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[414]	training's rmse: 0.787886	valid_1's rmse: 0.908751
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[415]	training's rmse: 0.787761	valid_1's rmse: 0.908734
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[416]	training's 

[508]	training's rmse: 0.777686	valid_1's rmse: 0.9085
[509]	training's rmse: 0.777625	valid_1's rmse: 0.908454
[510]	training's rmse: 0.777526	valid_1's rmse: 0.908405
[511]	training's rmse: 0.777425	valid_1's rmse: 0.908447
[512]	training's rmse: 0.777333	valid_1's rmse: 0.908496
[513]	training's rmse: 0.777266	valid_1's rmse: 0.908494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[514]	training's rmse: 0.777155	valid_1's rmse: 0.908583
[515]	training's rmse: 0.777044	valid_1's rmse: 0.908616
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[516]	training's rmse: 0.776989	valid_1's rmse: 0.908639
[517]	training's rmse: 0.776894	valid_1's rmse: 0.908688
[518]	training's rmse: 0.776815	valid_1's rmse: 0.908791
[519]	training's rmse: 0.776758	valid_1's rmse: 0.908772
[520]	training's rmse: 0.77666	valid_1's rmse: 0.908725
[521]	training's rmse: 0.77657	valid_1's rmse: 0.908779
[522]	training's rmse: 0.776478	valid_1's rmse: 0.908797

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1678]	training's rmse: 0.727916	valid_1's rmse: 0.897738
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1679]	training's rmse: 0.727894	valid_1's rmse: 0.897713
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1680]	training's rmse: 0.727875	valid_1's rmse: 0.897725
[1681]	training's rmse: 0.727852	valid_1's rmse: 0.897748
[1682]	training's rmse: 0.727827	valid_1's rmse: 0.897726
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1683]	training's rmse: 0.727814	valid_1's rmse: 0.897725
[1684]	training's rmse: 0.727794	valid_1's rmse: 0.897733
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1685]	training's rmse: 0.727764	valid_1's rmse: 0.897715
[1686]	training's rmse: 0.727742	valid_1's rmse: 0.897735
[1687]	training's rmse: 0.727726	valid_1's rmse: 0.89773
[LightGBM] [Warning] No further splits with po

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1756]	training's rmse: 0.726218	valid_1's rmse: 0.897351
[1757]	training's rmse: 0.726187	valid_1's rmse: 0.897287
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1758]	training's rmse: 0.726172	valid_1's rmse: 0.897292
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1759]	training's rmse: 0.726152	valid_1's rmse: 0.897279
[1760]	training's rmse: 0.726119	valid_1's rmse: 0.897308
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1761]	training's rmse: 0.726104	valid_1's rmse: 0.897316
[1762]	training's rmse: 0.726073	valid_1's rmse: 0.897273
[1763]	training's rmse: 0.726033	valid_1's rmse: 0.897268
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1764]	training's rmse: 0.726017	valid_1's rmse: 0.897189
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1765]	training's rmse: 0.72

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1832]	training's rmse: 0.724502	valid_1's rmse: 0.896772
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1833]	training's rmse: 0.72448	valid_1's rmse: 0.896786
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1834]	training's rmse: 0.72447	valid_1's rmse: 0.896784
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1835]	training's rmse: 0.724457	valid_1's rmse: 0.896785
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1836]	training's rmse: 0.724442	valid_1's rmse: 0.89679
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1837]	training's rmse: 0.724427	valid_1's rmse: 0.896796
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1838]	training's rmse: 0.724414	valid_1's rmse: 0.896795
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1902]	training's rmse: 0.723067	valid_1's rmse: 0.896398
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1903]	training's rmse: 0.723058	valid_1's rmse: 0.896396
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1904]	training's rmse: 0.72303	valid_1's rmse: 0.896422
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1905]	training's rmse: 0.723016	valid_1's rmse: 0.896417
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1906]	training's rmse: 0.723005	valid_1's rmse: 0.896415
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1907]	training's rmse: 0.722989	valid_1's rmse: 0.896399
[1908]	training's rmse: 0.722962	valid_1's rmse: 0.896476
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1909]	training's rmse: 0.722944	valid_1's rmse: 0.896469
[1910]	train

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1973]	training's rmse: 0.721758	valid_1's rmse: 0.896164
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1974]	training's rmse: 0.721753	valid_1's rmse: 0.896166
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1975]	training's rmse: 0.721737	valid_1's rmse: 0.896167
[1976]	training's rmse: 0.72171	valid_1's rmse: 0.89621
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1977]	training's rmse: 0.721695	valid_1's rmse: 0.896222
[1978]	training's rmse: 0.721667	valid_1's rmse: 0.896235
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1979]	training's rmse: 0.721655	valid_1's rmse: 0.896243
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1980]	training's rmse: 0.721637	valid_1's rmse: 0.896241
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1981]	traini

In [62]:
# 验证
from sklearn.metrics import mean_squared_error
valid_prediction = lgb_model.predict(X_valid).clip(0,20)
# valid_prediction
rmse_valid = np.sqrt(mean_squared_error(valid_prediction, label_valid))
rmse_valid

0.8959586618707074

In [69]:
rmse_valid = np.sqrt(mean_squared_error(valid_prediction, label_valid))
rmse_valid

0.8959586618707074

In [70]:
pred_33_df = pd.DataFrame({'item_cnt_month': valid_prediction})
dict(pred_33_df.item_cnt_month.value_counts())

{0.0: 52852,
 0.021887427341717867: 136,
 0.012392117780846399: 136,
 0.3455265551961666: 68,
 0.24419041349259255: 68,
 0.026545321919756892: 68,
 0.2367835393685169: 68,
 0.061109207249286694: 68,
 0.017461475290456933: 68,
 0.0810128069962931: 68,
 0.43670389976087154: 68,
 0.8244429340543955: 68,
 0.05607408554308525: 68,
 0.027647043693439094: 68,
 0.14090044646924477: 68,
 0.10104114505472708: 68,
 0.06546779812974014: 68,
 0.03748443870582796: 68,
 0.08008986438927498: 68,
 0.04923919402900914: 68,
 0.28538852346464577: 68,
 0.06145423313377659: 68,
 0.0014664723191730692: 68,
 0.0010629037175352758: 68,
 0.3231171078950427: 68,
 0.17296414273218086: 68,
 0.7684993299642364: 68,
 0.2541646160715933: 68,
 0.20708042053886874: 68,
 0.3420295705185275: 68,
 0.42265275413901376: 68,
 0.0302149066243018: 42,
 0.03971021618517349: 42,
 0.0058025753963794995: 40,
 0.12247429002485986: 36,
 0.00473559441640541: 36,
 0.1553274417725207: 34,
 0.0035624451123866864: 34,
 0.1458321322116491

In [71]:
# 对sub数据进行变换
def myfun(val):
    return round(val, 0)
pred_33_df['item_cnt_month'] = pred_33_df['item_cnt_month'].apply(myfun)

In [72]:
dict(pred_33_df.item_cnt_month.value_counts())

{0.0: 209542,
 1.0: 20638,
 2.0: 3907,
 3.0: 1334,
 4.0: 938,
 5.0: 707,
 6.0: 314,
 7.0: 171,
 8.0: 111,
 9.0: 76,
 10.0: 67,
 11.0: 61,
 13.0: 51,
 14.0: 47,
 15.0: 46,
 12.0: 44,
 19.0: 32,
 20.0: 22,
 16.0: 22,
 18.0: 22,
 17.0: 20}

In [73]:
m33 = matrix[matrix.date_block_num == 33]
dict(m33.item_cnt_month.value_counts())

{0.0: 206701,
 1.0: 21351,
 2.0: 5070,
 3.0: 1907,
 4.0: 959,
 5.0: 582,
 6.0: 341,
 20.0: 261,
 7.0: 226,
 8.0: 158,
 9.0: 138,
 10.0: 103,
 11.0: 65,
 12.0: 63,
 13.0: 54,
 15.0: 50,
 14.0: 46,
 16.0: 34,
 18.0: 25,
 17.0: 21,
 19.0: 17}

In [74]:
rmse_valid = np.sqrt(mean_squared_error(pred_33_df.item_cnt_month, label_valid))
rmse_valid

0.9143698651566013

# 训练33个月

In [75]:
matrix.shape

(6639294, 78)

In [76]:
"""建模"""
trainData = matrix[matrix['date_block_num'] < 34]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

train_data = lgb.Dataset(data=X_train, label=label_train)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 2000,
    'max_depth': 8,
    'num_leaves': 200,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.01,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data])

d:\program files\python37\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.338307 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13179
[LightGBM] [Info] Number of data points in the train set: 6425094, number of used features: 77
[LightGBM] [Info] Start training from score 0.287729
[1]	training's rmse: 1.18127
[2]	training's rmse: 1.17621
[3]	training's rmse: 1.17162
[4]	training's rmse: 1.16692
[5]	training's rmse: 1.16216
[6]	training's rmse: 1.15775
[7]	training's rmse: 1.15312
[8]	training's rmse: 1.14841
[9]	training's rmse: 1.144
[10]	training's rmse: 1.14047
[11]	training's rmse: 1.1364
[12]	training's rmse: 1.13244
[13]	training's rmse: 1.12785
[14]	training's rmse: 1.12393
[15]	training's rmse: 1.11936
[16]	training's rmse: 1.11626
[17]

[247]	training's rmse: 0.824517
[248]	training's rmse: 0.824202
[249]	training's rmse: 0.823775
[250]	training's rmse: 0.823128
[251]	training's rmse: 0.822741
[252]	training's rmse: 0.822433
[253]	training's rmse: 0.822065
[254]	training's rmse: 0.821852
[255]	training's rmse: 0.821606
[256]	training's rmse: 0.821143
[257]	training's rmse: 0.820666
[258]	training's rmse: 0.820423
[259]	training's rmse: 0.820208
[260]	training's rmse: 0.819948
[261]	training's rmse: 0.81973
[262]	training's rmse: 0.819541
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[263]	training's rmse: 0.81939
[264]	training's rmse: 0.818932
[265]	training's rmse: 0.818484
[266]	training's rmse: 0.818316
[267]	training's rmse: 0.818123
[268]	training's rmse: 0.817901
[269]	training's rmse: 0.817534
[270]	training's rmse: 0.817139
[271]	training's rmse: 0.816871
[272]	training's rmse: 0.816654
[273]	training's rmse: 0.816495
[274]	training's rmse: 0.816059
[275]	training's rmse: 0.815871

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[451]	training's rmse: 0.787285
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[452]	training's rmse: 0.787113
[453]	training's rmse: 0.787007
[454]	training's rmse: 0.786898
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[455]	training's rmse: 0.786844
[456]	training's rmse: 0.786713
[457]	training's rmse: 0.786626
[458]	training's rmse: 0.786489
[459]	training's rmse: 0.786299
[460]	training's rmse: 0.786214
[461]	training's rmse: 0.7861
[462]	training's rmse: 0.786008
[463]	training's rmse: 0.78591
[464]	training's rmse: 0.785838
[465]	training's rmse: 0.785715
[466]	training's rmse: 0.785609
[467]	training's rmse: 0.785511
[468]	training's rmse: 0.785396
[469]	training's rmse: 0.785277
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[470]	training's rmse: 0.78517
[471]	training's rmse: 0.784959
[472]	training's rmse: 0.784832


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[600]	training's rmse: 0.773954
[601]	training's rmse: 0.773897
[602]	training's rmse: 0.77381
[603]	training's rmse: 0.773728
[604]	training's rmse: 0.773652
[605]	training's rmse: 0.77358
[606]	training's rmse: 0.773537
[607]	training's rmse: 0.77347
[608]	training's rmse: 0.773374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[609]	training's rmse: 0.773308
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[610]	training's rmse: 0.773277
[611]	training's rmse: 0.773152
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[612]	training's rmse: 0.773098
[613]	training's rmse: 0.773035
[614]	training's rmse: 0.772924
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[615]	training's rmse: 0.772815
[616]	training's rmse: 0.772754
[617]	training's rmse: 0.772378
[618]	training's rmse: 0.772283
[619]	training's rms

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1577]	training's rmse: 0.735377
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1578]	training's rmse: 0.735358
[1579]	training's rmse: 0.735328
[1580]	training's rmse: 0.735293
[1581]	training's rmse: 0.735274
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1582]	training's rmse: 0.735259
[1583]	training's rmse: 0.735229
[1584]	training's rmse: 0.73521
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1585]	training's rmse: 0.735191
[1586]	training's rmse: 0.735171
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1587]	training's rmse: 0.735151
[1588]	training's rmse: 0.735124
[1589]	training's rmse: 0.735101
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1590]	training's rmse: 0.735079
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1591]	trainin

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1681]	training's rmse: 0.733065
[1682]	training's rmse: 0.73304
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1683]	training's rmse: 0.733024
[1684]	training's rmse: 0.732995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1685]	training's rmse: 0.732944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1686]	training's rmse: 0.732927
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1687]	training's rmse: 0.732916
[1688]	training's rmse: 0.732894
[1689]	training's rmse: 0.73287
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1690]	training's rmse: 0.732856
[1691]	training's rmse: 0.732818
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1692]	training's rmse: 0.732805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1693]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1789]	training's rmse: 0.730605
[1790]	training's rmse: 0.730578
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1791]	training's rmse: 0.730535
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1792]	training's rmse: 0.73052
[1793]	training's rmse: 0.730495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1794]	training's rmse: 0.730478
[1795]	training's rmse: 0.730461
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1796]	training's rmse: 0.730442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1797]	training's rmse: 0.730426
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1798]	training's rmse: 0.730418
[1799]	training's rmse: 0.730394
[1800]	training's rmse: 0.730366
[1801]	training's rmse: 0.73035
[LightGBM] [Warning] No further splits with posi

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1888]	training's rmse: 0.728598
[1889]	training's rmse: 0.728567
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1890]	training's rmse: 0.728557
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1891]	training's rmse: 0.728544
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1892]	training's rmse: 0.728533
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1893]	training's rmse: 0.728501
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1894]	training's rmse: 0.728486
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1895]	training's rmse: 0.728468
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1896]	training's rmse: 0.728454
[1897]	training's rmse: 0.728424
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[1976]	training's rmse: 0.726939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1977]	training's rmse: 0.72692
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1978]	training's rmse: 0.726909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1979]	training's rmse: 0.726893
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1980]	training's rmse: 0.726881
[1981]	training's rmse: 0.726863
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1982]	training's rmse: 0.726801
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1983]	training's rmse: 0.726789
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1984]	training's rmse: 0.72678
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1985]	training's rmse: 0.726735
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [77]:
six_zero = sales_by_item_id[(sales_by_item_id['28'] == 0) & (sales_by_item_id['29'] == 0) & (sales_by_item_id['30'] == 0) & (sales_by_item_id['31'] == 0) & (sales_by_item_id['32'] == 0) & (sales_by_item_id['33'] == 0)]
six_zero_item_id = list(six_zero['item_id'].values)   # item_id列表

In [78]:
# test数据
testData = matrix[matrix['date_block_num'] == 34]
X_test = testData.drop('item_cnt_month', axis=1)

# 预测&生成文件
y_test = lgb_model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})

test0 = test[test.item_id.isin(six_zero_item_id)]
ids = list(test0.ID.values)
submission.loc[submission.ID.isin(ids), 'item_cnt_month'] = 0.0
submission.to_csv('./submit/sub1.csv', index=False)

# 分析生成数据

In [90]:
sub = submission.copy()

In [91]:
dict(sub.item_cnt_month.value_counts())

{0.0: 40634,
 0.28960871983713027: 40,
 0.31306224439954544: 40,
 0.3257521661952946: 40,
 0.15455734424698034: 30,
 0.16724726604272952: 30,
 0.13214881600264755: 30,
 0.5606389593078824: 24,
 0.5925064195095856: 24,
 0.6051963413053348: 24,
 0.34484368881809035: 20,
 0.26730109839288346: 20,
 0.25611555617521714: 20,
 0.24444406949205283: 20,
 0.6116608274901834: 20,
 0.2941234411157476: 20,
 0.4518087967761352: 20,
 0.7033786974574552: 20,
 0.6571099795850913: 20,
 1.1167932283511597: 20,
 0.29029480931521773: 20,
 0.22429871211505492: 20,
 0.28277410421464017: 20,
 0.26998387608865454: 20,
 0.3676663355162458: 20,
 0.2794577127593258: 20,
 0.7766794935780167: 20,
 0.2716931616295741: 20,
 0.36219276657063737: 20,
 0.6072537884927857: 20,
 1.2919255699860102: 20,
 0.2805705466966578: 20,
 0.38925965048295186: 20,
 0.3542467262196439: 20,
 0.5986790215953899: 20,
 0.3191045477231047: 20,
 0.5083011027195751: 20,
 0.3651896882496883: 20,
 0.3884026834231535: 20,
 0.2689121668483746: 2

In [92]:
# 对sub数据进行变换

def myfun(val):
    return round(val, 0)
    
sub['item_cnt_month'] = sub['item_cnt_month'].apply(myfun)

In [93]:
dict(sub.item_cnt_month.value_counts())

{0.0: 183539,
 1.0: 22279,
 2.0: 3788,
 3.0: 1465,
 4.0: 1092,
 5.0: 749,
 6.0: 461,
 7.0: 179,
 8.0: 109,
 10.0: 75,
 9.0: 72,
 12.0: 71,
 15.0: 58,
 11.0: 51,
 14.0: 43,
 13.0: 37,
 16.0: 32,
 20.0: 28,
 19.0: 27,
 17.0: 26,
 18.0: 19}

In [94]:
sub.to_csv('./submit/sub1_1.csv', index=False)

In [80]:
m33 = matrix[matrix.date_block_num == 33]
dict(m33.item_cnt_month.value_counts())

{0.0: 206701,
 1.0: 21351,
 2.0: 5070,
 3.0: 1907,
 4.0: 959,
 5.0: 582,
 6.0: 341,
 20.0: 261,
 7.0: 226,
 8.0: 158,
 9.0: 138,
 10.0: 103,
 11.0: 65,
 12.0: 63,
 13.0: 54,
 15.0: 50,
 14.0: 46,
 16.0: 34,
 18.0: 25,
 17.0: 21,
 19.0: 17}

In [77]:
m32 = matrix[matrix.date_block_num == 32]
dict(m32.item_cnt_month.value_counts())

{0.0: 189052,
 1.0: 20128,
 2.0: 4810,
 3.0: 1777,
 4.0: 904,
 5.0: 525,
 6.0: 308,
 20.0: 268,
 7.0: 222,
 8.0: 137,
 9.0: 115,
 10.0: 81,
 11.0: 62,
 12.0: 56,
 13.0: 47,
 19.0: 31,
 16.0: 30,
 14.0: 29,
 15.0: 27,
 17.0: 24,
 18.0: 22}

In [78]:
m31 = matrix[matrix.date_block_num == 31]
dict(m31.item_cnt_month.value_counts())

{0.0: 181110,
 1.0: 22413,
 2.0: 5677,
 3.0: 2151,
 4.0: 1044,
 5.0: 683,
 6.0: 431,
 7.0: 255,
 8.0: 154,
 20.0: 123,
 9.0: 121,
 10.0: 92,
 11.0: 59,
 12.0: 55,
 13.0: 41,
 15.0: 30,
 16.0: 24,
 14.0: 22,
 17.0: 18,
 19.0: 17,
 18.0: 16}

In [79]:
m30 = matrix[matrix.date_block_num == 30]
dict(m30.item_cnt_month.value_counts())

{0.0: 195431,
 1.0: 22977,
 2.0: 5720,
 3.0: 2030,
 4.0: 964,
 5.0: 520,
 6.0: 326,
 7.0: 197,
 8.0: 143,
 20.0: 120,
 9.0: 111,
 10.0: 80,
 11.0: 56,
 12.0: 54,
 14.0: 39,
 13.0: 38,
 15.0: 21,
 16.0: 19,
 18.0: 18,
 17.0: 14,
 19.0: 11}

# 新的开始

In [185]:
matrix.shape

(11128050, 69)

In [186]:
"""建模"""
trainData = matrix[matrix['date_block_num'] < 33]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

validData = matrix[matrix['date_block_num'] == 33]
label_valid = validData['item_cnt_month']
X_valid = validData.drop('item_cnt_month', axis=1)

In [187]:
2**10

1024

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(data=X_train, label=label_train)
valid_data = lgb.Dataset(data=X_valid, label=label_valid)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 3000,
    'max_depth': 9,
    'num_leaves': 300,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.01,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data, valid_data])

[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.225803 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11641
[LightGBM] [Info] Number of data points in the train set: 10675678, number of used features: 68
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Info] Start training from score 0.299125
[1]	training's rmse: 1.2197	valid_1's rmse: 1.13357
[2]	training's rmse: 1.21523	valid_1's rmse: 1.13003
[3]	training's rmse: 1.2123	valid_1's rmse: 1.12753
[4]	training's rmse: 1.20782	valid_1's rmse: 1.12392
[5]	training's rmse: 1.20355	valid_1's rmse: 1.1208
[6]	training's rmse: 1.20017	valid_1's rmse: 1.11787
[7]	training's rmse: 1.19554	valid_1's rmse: 1.11409
[8]	training's rmse: 1.19168	valid_1's rmse: 

[140]	training's rmse: 0.94587	valid_1's rmse: 0.935335
[141]	training's rmse: 0.945163	valid_1's rmse: 0.935025
[142]	training's rmse: 0.94442	valid_1's rmse: 0.934555
[143]	training's rmse: 0.943888	valid_1's rmse: 0.934322
[144]	training's rmse: 0.943132	valid_1's rmse: 0.933978
[145]	training's rmse: 0.942442	valid_1's rmse: 0.933589
[146]	training's rmse: 0.941703	valid_1's rmse: 0.933268
[147]	training's rmse: 0.941064	valid_1's rmse: 0.933005
[148]	training's rmse: 0.940301	valid_1's rmse: 0.932469
[149]	training's rmse: 0.939496	valid_1's rmse: 0.932101
[150]	training's rmse: 0.938944	valid_1's rmse: 0.931827
[151]	training's rmse: 0.93827	valid_1's rmse: 0.931484
[152]	training's rmse: 0.937814	valid_1's rmse: 0.931154
[153]	training's rmse: 0.93726	valid_1's rmse: 0.930885
[154]	training's rmse: 0.936727	valid_1's rmse: 0.930696
[155]	training's rmse: 0.936121	valid_1's rmse: 0.93035
[156]	training's rmse: 0.935492	valid_1's rmse: 0.929886
[157]	training's rmse: 0.93478	valid

[285]	training's rmse: 0.89215	valid_1's rmse: 0.914237
[286]	training's rmse: 0.891981	valid_1's rmse: 0.9142
[287]	training's rmse: 0.891635	valid_1's rmse: 0.914125
[288]	training's rmse: 0.891253	valid_1's rmse: 0.914136
[289]	training's rmse: 0.891089	valid_1's rmse: 0.914158
[290]	training's rmse: 0.890914	valid_1's rmse: 0.914127
[291]	training's rmse: 0.890675	valid_1's rmse: 0.914045
[292]	training's rmse: 0.890532	valid_1's rmse: 0.914027
[293]	training's rmse: 0.890376	valid_1's rmse: 0.914004
[294]	training's rmse: 0.890205	valid_1's rmse: 0.914008
[295]	training's rmse: 0.889996	valid_1's rmse: 0.913917
[296]	training's rmse: 0.88988	valid_1's rmse: 0.913947
[297]	training's rmse: 0.889695	valid_1's rmse: 0.913946
[298]	training's rmse: 0.889546	valid_1's rmse: 0.913966
[299]	training's rmse: 0.889369	valid_1's rmse: 0.913925
[300]	training's rmse: 0.889201	valid_1's rmse: 0.913887
[301]	training's rmse: 0.889013	valid_1's rmse: 0.914008
[302]	training's rmse: 0.888718	val

[430]	training's rmse: 0.870925	valid_1's rmse: 0.911865
[431]	training's rmse: 0.870826	valid_1's rmse: 0.91177
[432]	training's rmse: 0.870723	valid_1's rmse: 0.911756
[433]	training's rmse: 0.870473	valid_1's rmse: 0.911617
[434]	training's rmse: 0.87035	valid_1's rmse: 0.911654
[435]	training's rmse: 0.870195	valid_1's rmse: 0.911639
[436]	training's rmse: 0.870103	valid_1's rmse: 0.911612
[437]	training's rmse: 0.870043	valid_1's rmse: 0.911611
[438]	training's rmse: 0.86995	valid_1's rmse: 0.911603
[439]	training's rmse: 0.86984	valid_1's rmse: 0.911549
[440]	training's rmse: 0.869756	valid_1's rmse: 0.911588
[441]	training's rmse: 0.869638	valid_1's rmse: 0.911582
[442]	training's rmse: 0.869591	valid_1's rmse: 0.911587
[443]	training's rmse: 0.869483	valid_1's rmse: 0.911546
[444]	training's rmse: 0.869417	valid_1's rmse: 0.911522
[445]	training's rmse: 0.869368	valid_1's rmse: 0.911514
[446]	training's rmse: 0.869299	valid_1's rmse: 0.911532
[447]	training's rmse: 0.869224	val

[563]	training's rmse: 0.858673	valid_1's rmse: 0.910156
[564]	training's rmse: 0.858594	valid_1's rmse: 0.910169
[565]	training's rmse: 0.858526	valid_1's rmse: 0.910146
[566]	training's rmse: 0.858467	valid_1's rmse: 0.910154
[567]	training's rmse: 0.858357	valid_1's rmse: 0.910135
[568]	training's rmse: 0.8583	valid_1's rmse: 0.910123
[569]	training's rmse: 0.858238	valid_1's rmse: 0.910103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[570]	training's rmse: 0.858198	valid_1's rmse: 0.91012
[571]	training's rmse: 0.858143	valid_1's rmse: 0.910123
[572]	training's rmse: 0.858101	valid_1's rmse: 0.910113
[573]	training's rmse: 0.85806	valid_1's rmse: 0.910105
[574]	training's rmse: 0.857988	valid_1's rmse: 0.910095
[575]	training's rmse: 0.857905	valid_1's rmse: 0.910137
[576]	training's rmse: 0.857846	valid_1's rmse: 0.910144
[577]	training's rmse: 0.857793	valid_1's rmse: 0.910153
[578]	training's rmse: 0.857665	valid_1's rmse: 0.910144
[579]	training's 

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2309]	training's rmse: 0.807658	valid_1's rmse: 0.906231
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2310]	training's rmse: 0.807647	valid_1's rmse: 0.906231
[2311]	training's rmse: 0.807632	valid_1's rmse: 0.906226
[2312]	training's rmse: 0.807598	valid_1's rmse: 0.906198
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2313]	training's rmse: 0.807582	valid_1's rmse: 0.906209
[2314]	training's rmse: 0.807565	valid_1's rmse: 0.906211
[2315]	training's rmse: 0.807539	valid_1's rmse: 0.906202
[2316]	training's rmse: 0.80749	valid_1's rmse: 0.906218
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2317]	training's rmse: 0.807479	valid_1's rmse: 0.906221
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2318]	training's rmse: 0.807468	valid_1's rmse: 0.906219
[2319]	training's rmse: 0.807445	valid_1's rms

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2388]	training's rmse: 0.806497	valid_1's rmse: 0.906023
[2389]	training's rmse: 0.80647	valid_1's rmse: 0.906059
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2390]	training's rmse: 0.806459	valid_1's rmse: 0.906054
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2391]	training's rmse: 0.806451	valid_1's rmse: 0.906051
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2392]	training's rmse: 0.806446	valid_1's rmse: 0.90603
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2393]	training's rmse: 0.806439	valid_1's rmse: 0.906061
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2394]	training's rmse: 0.806426	valid_1's rmse: 0.906079
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2395]	training's rmse: 0.806417	valid_1's rmse: 0.906074
[2396]	traini

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2464]	training's rmse: 0.805395	valid_1's rmse: 0.905911
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2465]	training's rmse: 0.805385	valid_1's rmse: 0.905913
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2466]	training's rmse: 0.805368	valid_1's rmse: 0.905935
[2467]	training's rmse: 0.805352	valid_1's rmse: 0.905945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2468]	training's rmse: 0.805343	valid_1's rmse: 0.905939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2469]	training's rmse: 0.805332	valid_1's rmse: 0.905937
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2470]	training's rmse: 0.805323	valid_1's rmse: 0.905951
[2471]	training's rmse: 0.805281	valid_1's rmse: 0.90595
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2472]	train

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2536]	training's rmse: 0.804446	valid_1's rmse: 0.905832
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2537]	training's rmse: 0.804436	valid_1's rmse: 0.90583
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2538]	training's rmse: 0.804429	valid_1's rmse: 0.905825
[2539]	training's rmse: 0.804413	valid_1's rmse: 0.905826
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2540]	training's rmse: 0.804408	valid_1's rmse: 0.905825
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2541]	training's rmse: 0.804397	valid_1's rmse: 0.905822
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2542]	training's rmse: 0.80439	valid_1's rmse: 0.905818
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2543]	training's rmse: 0.804379	valid_1's rmse: 0.905826
[LightGBM] [W

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2605]	training's rmse: 0.803595	valid_1's rmse: 0.905693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2606]	training's rmse: 0.803587	valid_1's rmse: 0.905691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2607]	training's rmse: 0.80358	valid_1's rmse: 0.905689
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2608]	training's rmse: 0.803576	valid_1's rmse: 0.905692
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2609]	training's rmse: 0.803567	valid_1's rmse: 0.905692
[2610]	training's rmse: 0.803553	valid_1's rmse: 0.905687
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2611]	training's rmse: 0.803543	valid_1's rmse: 0.905686
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2612]	training's rmse: 0.803538	valid_1's rmse: 0.905685
[LightGBM] [

In [139]:
valid_prediction = lgb_model.predict(X_valid).clip(0,20)

In [140]:
rmse_valid = np.sqrt(mean_squared_error(valid_prediction, label_valid))
rmse_valid

0.8986837882141382

In [141]:
pred_33_df = pd.DataFrame({'item_cnt_month': valid_prediction})
# 对sub数据进行变换
def myfun(val):
    return round(val, 0)
pred_33_df['item_cnt_month'] = pred_33_df['item_cnt_month'].apply(myfun)

In [142]:
rmse_valid = np.sqrt(mean_squared_error(pred_33_df['item_cnt_month'], label_valid))
rmse_valid

0.914353793551493

In [143]:
# 训练33个月数据
trainData = matrix[matrix['date_block_num'] < 34]
label_train = trainData['item_cnt_month']
X_train = trainData.drop('item_cnt_month', axis=1)

train_data = lgb.Dataset(data=X_train, label=label_train)
params = {
    'objective': 'regression',  # 回归
    'metric': 'rmse',   # 回归问题选择rmse
    'n_estimators': 400,
    'max_depth': 10,
    'num_leaves': 400,   # 每个弱学习器拥有的叶子的数量
    'learning_rate': 0.01,
    'bagging_fraction': 0.9,    # 每次训练“弱学习器”用的数据比例（应该也是随机的），用于加快训练速度和减小过拟合
    'feature_fraction': 0.3,   # 每次迭代过程中，随机选择30%的特征建树（弱学习器）
    'bagging_seed': 0,
    'early_stop_rounds': 50
}
lgb_model = lgb.train(params, train_data, valid_sets=[train_data])

d:\program files\python37\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Unknown parameter: early_stop_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.202945 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12627
[LightGBM] [Info] Number of data points in the train set: 6425094, number of used features: 73
[LightGBM] [Info] Start training from score 0.287729
[1]	training's rmse: 1.18113
[2]	training's rmse: 1.17602
[3]	training's rmse: 1.17077
[4]	training's rmse: 1.16592
[5]	training's rmse: 1.16092
[6]	training's rmse: 1.15622
[7]	training's rmse: 1.15128
[8]	training's rmse: 1.14652
[9]	training's rmse: 1.14165
[10]	training's rmse: 1.13707
[11]	training's rmse: 1.13277
[12]	training's rmse: 1.12832
[13]	training's rmse: 1.12401
[14]	training's rmse: 1.11985
[15]	training's rmse: 1.11573
[16]	training's rmse: 1.11259
[

[247]	training's rmse: 0.814577
[248]	training's rmse: 0.814312
[249]	training's rmse: 0.814009
[250]	training's rmse: 0.813757
[251]	training's rmse: 0.813546
[252]	training's rmse: 0.813333
[253]	training's rmse: 0.813041
[254]	training's rmse: 0.812772
[255]	training's rmse: 0.812545
[256]	training's rmse: 0.812265
[257]	training's rmse: 0.812044
[258]	training's rmse: 0.811576
[259]	training's rmse: 0.811284
[260]	training's rmse: 0.810965
[261]	training's rmse: 0.810565
[262]	training's rmse: 0.8103
[263]	training's rmse: 0.809949
[264]	training's rmse: 0.809749
[265]	training's rmse: 0.809455
[266]	training's rmse: 0.80916
[267]	training's rmse: 0.808934
[268]	training's rmse: 0.808656
[269]	training's rmse: 0.808324
[270]	training's rmse: 0.807968
[271]	training's rmse: 0.807654
[272]	training's rmse: 0.807468
[273]	training's rmse: 0.807282
[274]	training's rmse: 0.806939
[275]	training's rmse: 0.806729
[276]	training's rmse: 0.806516
[277]	training's rmse: 0.80621
[278]	traini

In [145]:
# test数据
testData = matrix[matrix['date_block_num'] == 34]
X_test = testData.drop('item_cnt_month', axis=1)

# 预测&生成文件
y_test = lgb_model.predict(X_test).clip(0, 20)
submission = pd.DataFrame({ 'ID': range(0, 214200), 'item_cnt_month': y_test})

test0 = test[test.item_id.isin(six_zero_item_id)]
ids = list(test0.ID.values)
submission.loc[submission.ID.isin(ids), 'item_cnt_month'] = 0.0
submission.to_csv('./submit/sub3.csv', index=False)